## Load Training / Testing Data

In [1]:
FileSaveFolder = "A:\\Files\\Shares\\Downloads\\yelp_dataset\\yelp_dataset~\\"
def LoadTrainTestData():
    import pickle
    with open(FileSaveFolder + "TrainTestData.dat", "rb") as filePath:
        train_x = pickle.load(file=filePath)
        train_y = pickle.load(file=filePath)
        test_x = pickle.load(file=filePath)
        test_y = pickle.load(file=filePath)
    return train_x, train_y, test_x, test_y

In [2]:
Train_X, Train_Y, Test_X, Test_Y = LoadTrainTestData()

In [3]:
print(Train_X.shape)
print(Train_Y.shape)
print(Test_X.shape)
print(Test_Y.shape)
print(Test_Y.unique())

(151483, 1743)
(151483,)
(37871, 1743)
(37871,)
[2.  4.  3.  2.5 3.5]


### Trialing different XGB parameters

In [3]:
import pandas as pd
XGBResults = pd.DataFrame()

def AssessXGBRegression(x_train, y_train, x_test, y_test, Model):
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    from datetime import datetime

    loopStartTime = datetime.now()
    print("Starting XGBRegressor at " + str(loopStartTime))

    Model.fit(x_train, y_train)
    xgbPredictor = Model.predict(x_test)

    loopEndTime = datetime.now()
    # results container
    results_list = pd.DataFrame(data = {
        "Name" : "XGBRegressor",
        "R2Score": r2_score(y_test, xgbPredictor),
        "RMSE": (mean_squared_error(y_test, xgbPredictor)),
        "MAE": mean_absolute_error(y_test, xgbPredictor),
        "ModelData" : Model,
        "RunTime": [str(loopEndTime - loopStartTime)],
        })

    print("\tEnding XGBRegressor at " + str(datetime.now()))

    return results_list

In [ ]:
import pandas as pd
import copy

#for maxDepth in [6, 9, 12]:
for maxDepth in [12, 15]:
    import gc
    gc.collect()
    for learningRate in [0.01, 0.05, 0.1]:
#    for learningRate in [0.33, 0.5]:
        from xgboost import XGBRegressor
        import warnings
        warnings.simplefilter(action='ignore', category=FutureWarning)
#        for estimatorsCount in [100, 200, 400, 800]:
        for estimatorsCount in [800, 1600]:
        #estimatorsCount = 50 / learningRate

            xgbModel = XGBRegressor(objective = "reg:squarederror",
                                    colsample_bytree = 1,
                                    colsample_bylevel = 1,
                                    colsample_bynode = 1,
                                    learning_rate = learningRate,
                                    max_depth = maxDepth,
                                    tree_method = "hist",
                                    grow_policy = "lossguide",
                                    n_estimators = estimatorsCount,
                                    nthread = 6,
                                   )

            testTrainResults = AssessXGBRegression(Train_X, Train_Y, Test_X, Test_Y, Model = xgbModel)
            testTrainResults["LearnRate"] = learningRate
            testTrainResults["MaxDepth"] = maxDepth
            testTrainResults["Estimators"] = estimatorsCount

            with pd.option_context("display.max_rows", None, "display.max_columns", None, "display.max_colwidth", 20):
                display(testTrainResults.drop(["Name", "ModelData"], axis = 1))

            XGBResults = XGBResults.append(copy.deepcopy(testTrainResults), ignore_index = True)

            del xgbModel
            del testTrainResults
            gc.collect()

print("Loops Complete.")

Starting XGBRegressor at 2020-01-28 00:09:06.866781


In [5]:
XGBResults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
Name          8 non-null object
R2Score       8 non-null float64
RMSE          8 non-null float64
MAE           8 non-null float64
ModelData     8 non-null object
RunTime       8 non-null object
LearnRate     8 non-null float64
MaxDepth      8 non-null int64
Estimators    8 non-null int64
dtypes: float64(4), int64(2), object(3)
memory usage: 704.0+ bytes


In [4]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,-1.295799,1.266481,0.957040,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:53.815709,0.01,6,100
1,XGBRegressor,0.200750,0.440907,0.555921,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:24.839741,0.01,6,200
2,XGBRegressor,0.508864,0.270936,0.428631,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:27.005766,0.01,6,400
3,XGBRegressor,0.581160,0.231054,0.387177,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:04:18.020098,0.01,6,800
4,XGBRegressor,0.538833,0.254403,0.411359,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:47.965588,0.05,6,100
5,XGBRegressor,0.599257,0.221070,0.376750,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:15.937601,0.05,6,200
6,XGBRegressor,0.638384,0.199486,0.354655,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:10.311130,0.05,6,400
7,XGBRegressor,0.664054,0.185325,0.340152,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:04:00.164835,0.05,6,800
8,XGBRegressor,0.598448,0.221517,0.376770,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:48.684357,0.10,6,100
9,XGBRegressor,0.637465,0.199993,0.354905,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:16.094550,0.10,6,200


In [6]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,0.642602,0.197159,0.351341,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:45.457394,0.33,6,100
1,XGBRegressor,0.659730,0.187710,0.341321,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:10.456362,0.33,6,200
2,XGBRegressor,0.665457,0.184551,0.337758,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:57.484249,0.33,6,400
3,XGBRegressor,0.665503,0.184526,0.337621,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:24.750214,0.33,6,800
4,XGBRegressor,0.636700,0.200415,0.353084,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:44.509698,0.50,6,100
5,XGBRegressor,0.642804,0.197048,0.349631,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:08.902862,0.50,6,200
6,XGBRegressor,0.641120,0.197977,0.350026,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:57.406278,0.50,6,400
7,XGBRegressor,0.635334,0.201169,0.352150,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:23.678557,0.50,6,800


In [5]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,-1.239562,1.235457,0.954942,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:42.850954,0.01,9,100
1,XGBRegressor,0.258766,0.408903,0.529119,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:12.793055,0.01,9,200
2,XGBRegressor,0.560077,0.242684,0.398456,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:05:50.837234,0.01,9,400
3,XGBRegressor,0.620921,0.209120,0.362775,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:10:36.213583,0.01,9,800
4,XGBRegressor,0.585563,0.228624,0.383543,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:38.397384,0.05,9,100
5,XGBRegressor,0.633914,0.201952,0.355232,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:46.739426,0.05,9,200
6,XGBRegressor,0.663110,0.185846,0.338549,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:05:05.628800,0.05,9,400
7,XGBRegressor,0.682078,0.175382,0.327309,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:09:36.930630,0.05,9,800
8,XGBRegressor,0.632436,0.202767,0.355680,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:34.519630,0.10,9,100
9,XGBRegressor,0.660056,0.187531,0.340118,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:47.567159,0.10,9,200


In [6]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,0.645952,0.195311,0.346346,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:27.228973,0.33,9,100
1,XGBRegressor,0.653447,0.191176,0.341744,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:27.903478,0.33,9,200
2,XGBRegressor,0.653301,0.191257,0.341347,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:04:10.221604,0.33,9,400
3,XGBRegressor,0.649908,0.193129,0.342928,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:07:08.588294,0.33,9,800
4,XGBRegressor,0.621275,0.208924,0.357720,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:23.884047,0.50,9,100
5,XGBRegressor,0.616739,0.211427,0.358951,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:21.570513,0.50,9,200
6,XGBRegressor,0.608165,0.216157,0.362979,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:59.405078,0.50,9,400
7,XGBRegressor,0.598624,0.221419,0.367071,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:07:05.547271,0.50,9,800


In [6]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,0.645952,0.195311,0.346346,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:27.935744,0.33,9,100
1,XGBRegressor,0.649908,0.193129,0.342928,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:07:08.410350,0.33,9,800
2,XGBRegressor,0.621275,0.208924,0.357720,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:24.060991,0.50,9,100
3,XGBRegressor,0.598624,0.221419,0.367071,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:07:02.369292,0.50,9,800
4,XGBRegressor,0.640007,0.198591,0.347138,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:01.709616,0.33,12,100
5,XGBRegressor,0.636096,0.200748,0.348415,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:15:07.315477,0.33,12,800
6,XGBRegressor,0.600747,0.220248,0.364892,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:55.548595,0.50,12,100
7,XGBRegressor,0.584032,0.229469,0.372324,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:16:06.669406,0.50,12,800


In [7]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,-1.202747,1.215148,0.953534,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:16.880753,0.01,12,100
1,XGBRegressor,0.296838,0.387900,0.511755,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:07:05.148399,0.01,12,200
2,XGBRegressor,0.590516,0.225892,0.379642,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:13:47.017277,0.01,12,400
3,XGBRegressor,0.641709,0.197651,0.349214,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:24:59.704141,0.01,12,800
4,XGBRegressor,0.611202,0.214481,0.367063,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:34.268155,0.05,12,100
5,XGBRegressor,0.650275,0.192926,0.343891,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:06:19.793971,0.05,12,200
6,XGBRegressor,0.672559,0.180633,0.331131,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:11:43.366006,0.05,12,400
7,XGBRegressor,0.685293,0.173608,0.323478,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:21:24.044433,0.05,12,800
8,XGBRegressor,0.646101,0.195229,0.346033,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:20.301643,0.10,12,100
9,XGBRegressor,0.666637,0.183900,0.334258,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:06:02.821425,0.10,12,200


In [6]:
XGBResults

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
0,XGBRegressor,0.640007,0.198591,0.347138,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:03:02.094492,0.33,12,100
1,XGBRegressor,0.639895,0.198652,0.347012,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:05:07.906068,0.33,12,200
2,XGBRegressor,0.638734,0.199293,0.347247,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:08:26.237344,0.33,12,400
3,XGBRegressor,0.636096,0.200748,0.348415,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:15:10.529444,0.33,12,800
4,XGBRegressor,0.600747,0.220248,0.364892,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:02:52.951430,0.50,12,100
5,XGBRegressor,0.595101,0.223363,0.367512,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:04:49.231070,0.50,12,200
6,XGBRegressor,0.588076,0.227239,0.370620,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:08:30.889850,0.50,12,400
7,XGBRegressor,0.584032,0.229469,0.372324,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:16:17.513922,0.50,12,800


In [6]:
import pickle
import pandas as pd

XGBAggregateResults = pd.DataFrame()
for i in range(1,11):
    with open(FileSaveFolder + "XGBRegressorResults{}of8.dat".format(str(i)), "rb") as filePath:
        XGBAggregateResults = XGBAggregateResults.append(pickle.load(file=filePath), ignore_index = True)

In [7]:
XGBAggregateResults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 9 columns):
Name          84 non-null object
R2Score       84 non-null float64
RMSE          84 non-null float64
MAE           84 non-null float64
ModelData     84 non-null object
RunTime       84 non-null object
LearnRate     84 non-null float64
MaxDepth      84 non-null int64
Estimators    84 non-null int64
dtypes: float64(4), int64(2), object(3)
memory usage: 6.0+ KB


In [13]:
XGBAggregateResults[XGBAggregateResults.duplicated(subset = ["LearnRate", "MaxDepth", "Estimators"], keep = False)].sort_values(["LearnRate", "MaxDepth", "Estimators"])

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
55,XGBRegressor,0.641709,0.197651,0.349214,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:24:59.704141,0.01,12,800
72,XGBRegressor,0.641709,0.197651,0.349214,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:25:15.589038,0.01,12,800
59,XGBRegressor,0.685293,0.173608,0.323478,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:21:24.044433,0.05,12,800
74,XGBRegressor,0.685293,0.173608,0.323478,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:21:45.547796,0.05,12,800
63,XGBRegressor,0.683002,0.174872,0.324539,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:19:00.616516,0.10,12,800
76,XGBRegressor,0.683002,0.174872,0.324539,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:19:25.449250,0.10,12,800
12,XGBRegressor,0.642602,0.197159,0.351341,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:45.136497,0.33,6,100
14,XGBRegressor,0.642602,0.197159,0.351341,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:00:45.457394,0.33,6,100
13,XGBRegressor,0.659730,0.187710,0.341321,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:10.517343,0.33,6,200
15,XGBRegressor,0.659730,0.187710,0.341321,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:01:10.456362,0.33,6,200


In [14]:
XGBAggregateResults.drop_duplicates(subset = ["LearnRate", "MaxDepth", "Estimators"], inplace = True)

In [15]:
XGBAggregateResults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 83
Data columns (total 9 columns):
Name          69 non-null object
R2Score       69 non-null float64
RMSE          69 non-null float64
MAE           69 non-null float64
ModelData     69 non-null object
RunTime       69 non-null object
LearnRate     69 non-null float64
MaxDepth      69 non-null int64
Estimators    69 non-null int64
dtypes: float64(4), int64(2), object(3)
memory usage: 5.4+ KB


In [16]:
import pickle
with open(FileSaveFolder + "XGBRegressorResults-All.dat", "wb") as filePath:
    pickle.dump(XGBAggregateResults, file=filePath)

In [16]:
import pickle
with open(FileSaveFolder + "XGBRegressorResults-All.dat", "rb") as filePath:
    XGBAggregateResults = pickle.load(file=filePath)

In [17]:
XGBAggregateResults[XGBAggregateResults["R2Score"] == XGBAggregateResults["R2Score"].max()]

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
75,XGBRegressor,0.690969,0.170477,0.319711,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:37:50.588825,0.05,12,1600


In [18]:
XGBAggregateResults[XGBAggregateResults["RMSE"] == XGBAggregateResults["RMSE"].min()]

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
75,XGBRegressor,0.690969,0.170477,0.319711,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:37:50.588825,0.05,12,1600


In [19]:
XGBAggregateResults[XGBAggregateResults["MAE"] == XGBAggregateResults["MAE"].min()]

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
75,XGBRegressor,0.690969,0.170477,0.319711,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:37:50.588825,0.05,12,1600


In [20]:
XGBAggregateResults.sort_values("R2Score", ascending = False).head(5)

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
75,XGBRegressor,0.690969,0.170477,0.319711,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:37:50.588825,0.05,12,1600
33,XGBRegressor,0.687139,0.172590,0.324128,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:08:52.154011,0.10,9,800
81,XGBRegressor,0.685652,0.173410,0.321714,"XGBRegressor(base_score=0.5, booster='gbtree',...",1:09:40.021949,0.05,15,1600
59,XGBRegressor,0.685293,0.173608,0.323478,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:21:24.044433,0.05,12,800
77,XGBRegressor,0.684057,0.174290,0.323933,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:32:31.744096,0.10,12,1600


In [21]:
XGBAggregateResults.sort_values("RMSE", ascending = True).head(5)

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
75,XGBRegressor,0.690969,0.170477,0.319711,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:37:50.588825,0.05,12,1600
33,XGBRegressor,0.687139,0.172590,0.324128,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:08:52.154011,0.10,9,800
81,XGBRegressor,0.685652,0.173410,0.321714,"XGBRegressor(base_score=0.5, booster='gbtree',...",1:09:40.021949,0.05,15,1600
59,XGBRegressor,0.685293,0.173608,0.323478,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:21:24.044433,0.05,12,800
77,XGBRegressor,0.684057,0.174290,0.323933,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:32:31.744096,0.10,12,1600


In [22]:
XGBAggregateResults.sort_values("MAE", ascending = True).head(5)

,Name,R2Score,RMSE,MAE,ModelData,RunTime,LearnRate,MaxDepth,Estimators
75,XGBRegressor,0.690969,0.170477,0.319711,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:37:50.588825,0.05,12,1600
81,XGBRegressor,0.685652,0.173410,0.321714,"XGBRegressor(base_score=0.5, booster='gbtree',...",1:09:40.021949,0.05,15,1600
59,XGBRegressor,0.685293,0.173608,0.323478,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:21:24.044433,0.05,12,800
80,XGBRegressor,0.682502,0.175148,0.323666,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:41:08.684970,0.05,15,800
77,XGBRegressor,0.684057,0.174290,0.323933,"XGBRegressor(base_score=0.5, booster='gbtree',...",0:32:31.744096,0.10,12,1600


In [23]:
XGBAggregateResults["ModelData"][75]

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             grow_policy='lossguide', importance_type='gain',
             learning_rate=0.05, max_delta_step=0, max_depth=12,
             min_child_weight=1, missing=nan, n_estimators=1600, n_jobs=1,
             nthread=6, objective='reg:squarederror', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, tree_method='hist', verbosity=1)

In [24]:
import pickle
with open("XGBRegressorResults.dat", "wb") as filePath:
    pickle.dump(XGBAggregateResults["ModelData"][75], file=filePath)